# mysql数据库笔记

## 1.mysql概述

Mysql是最流行的关系型数据库管理系统，在Web应用方面是最好的RDBMS(Relational Database Management System)。

***RDBMS的一些术语***

***数据库***：数据库是一些关联表表的集合。

***数据表***：表是数据的矩阵，在一个数据库中的表看起来像一个简单的电子表格。

***列***：一列（数据元素）包含了相同的数据，例如邮政编码的数据

***行***：一行（=元组，或记录）是一组相关的数据，例如一条用户订阅的数据

***主键***：主键是唯一的，一个数据表中只能包含一个主键，你可以使用主键来查询数据。

***外键***：外键用于关联两个表

***符合键***：复合键（组合键）将多个列作为一个索引键，一般用于符合索引。

***索引***：使用索引可快速访问数据库表中的特定信息。索引是对数据库表中一类或多列的值进行排序的一种结构，类似于数据的目录。

***特点***

支持开源的；

支持大型数据库，可以处理拥有上千万条记录的大型数据库；

使用标准的SQL数据语言形式；

可以应用于多个操作系统，支持多种语言；

## 2.mysql的安装卸载及编码问题等

sudo apt install mysql-server  # 服务端

sudo apt install mysql-server  # 客户端

netstat -antp |grep 3306  # 查看3306的端口

ps aux | grep mysql  # 查看mysql的进程

### 1.允许远程访问设置

***mysql设置root用户密码***

***方法1***：用set password命令
首先登录mysql

格式：mysql> set password for 用户名@localhost =password('新密码')；

例子：mysql> set password for root@localhost = password('123');

***方法2***：用update直接编辑user表

首先登录mysql
mysql>use mysql;
mysql>update user set password=password('123') where user='root' and host='localhost';
mysql>flush privileges;

***为所有登录机器授权***

mysql>grant all privileges on *.* to 'root'@'%' identified by 'root'

*.*表示所有库的所有对象（表、视图等）

identified by 'root用户的口令' 
mysql>flush privileges  权限立即生效


***修改配置文件中bind-address为mysql服务器的ip地址***

sudo vi /etc/mysql/mysql.conf.d/mysqld.cnf

修改bind-adderss=10.35.163.40   修改为自己的ip

***重启mysql服务***

sudo service mysql restart

### 2.解决中文的乱码问题

当创建了xxx数据库，创建了xxx表，表中有varchar或者char字符集，往表中插入数据时，输入中文会表报错（编码不对），这时需要：

***修改mysql server 的配置文件***

sudo vi /etc/mysql/mysql.conf.d/mysqld.cnf

在【mysqld】的下方添加 character-set-server = utf8


***修改mysql client 的配置文件***

假如在sudo vi /etc/mysql/conf.d/mysql.cnf

在【mysql】下方添加character-set-server = utf8 会造成mysql不能输入中文

***重启mysql服务***

sudo service mysql restart 

这时之前创建的文件添加中文依然会报错，需要对表中数据的编码进行修改

***修改创建的非utf8编码的表问题***

show create table <表名>

1.修改表字符集

mysql>alter table <表名> default charset=utf8;


2.修改表字段字符集

mysql>alter table <表名> modify <列名> 类型（长度）character set utf8;



### 3.重装mysql过程

***1.停止mysql服务***

sudo service mysql stop

***2.删除server和client***

sudo apt autoremove --purge mysql-server-5.5

sudo apt autotemove mysql-client

sudo apt remove mysql-common

***3.清理残留数据***

dpkg -l| grep ^rc | awk '{print $2}' |sudo xargs dpkg -p

***4.重新安装***

sudo apt-get install mysql-server


### 4.进入mysql环境

mysql 第一次使用

进入后建议修改root的口令

use mysql

update  user set authentication_string=password("root") where user='root';

mysql -uroot -p

输入密码

以root用户身份进入

### 5.数据库的创建、修改与删除

show databases       # 查看所有的数据库

create database pys    # 创建数据库的pys

drop database pys     # 删除数据库的pys

use pys            # 打开pys数据库

***数据类型***

varchar 可变长度数据类型，使用时必须指定长度

int  整型，默认的长度为11位，并不是指数据的位数

float  小数型，除了长度之外，可以指定小数的位数

date/datestamp  日期或日期时间类型

text  多行文本类型

enum  枚举类型，指定列值的范围

json json 数据类型  java脚本对象的数据

### 6.表的创建、删除

show tables  # 查看数据库中的表

create table 表名(字段名1 类型（数据长度）[约束] [comment '列注释']，列名2……)

drop table  <表名>   # 删除表名

desc <表名>        # 查看表结构

show create table 表名  # 查看创建表的sql语句

***修改表名***

alter table oldtable rename newtable;    # 将表名oldtable改为 newtable

***修改列名***

alter table fruit change name fname  varchar(32) not null; # 修改fruit表中name列的列名为fname  

***修改列结构***

alter table fruit modify name varchar(20) not null;  # 修改fruit表中的name变量的结构

***添加列***
alter table fruit add cnt int(3) default 100;  # 添加一个新的列

插入新列时，新列有默认值，前面的数据会自动添加设置的默认值；如果是已有的列，设置了默认值，默认值不会修改之前的记录

当插入数据时，会默认使用之前设置的默认值，如果是使用默认值得话，会采用你设置的值

*** 删除列***
alter table 表名 drop [column] 列名


### 7.数据的insert /select/update/delete

***insert***

insert into 表名 [(列名1[,列名2……])] values (列值1[,列值2])；

如果列名都不写，则会为所有列添加值

insert into 表名 values (列值1，列值2，列值3)；

批量插入

insert into 表名 values (列值1，列值2),(列值1，列值2)；

***select***

格:1：select 字段名,字段名2 from 表名 
	[where 条件］[order by 字段名 [AES｜DESC]] [group by 字段名 [having 条件]]  
    
格式2：select * from 表名 [where 条件]   查询所有记录

格式3：select 字段1，字段2 from 表名1  join 表名2 on 连接条件 [where 条件]

between 起始值 and 结束值


select * from f_order where create_date between "2018-04-18 16:00:00" and "2018-04-18 21:00:00";

like 模糊查询  % 任意多个字符  _任意一个字符

select * from f_order where create_date like '%-18%';

in()  范围查询

***扩展***

create view 视图名

select * from 视图名

***update***

update 表名 set 字段1=字段值，字段2=字段值  [where 条件]


***delete***

delete from 表名 [where 条件]

### 8.约束

五种约束：primary key  主键（唯一且不为空）

foreign key  外键（关联外表中的主键值，可以为空）

unique  唯一

not null 不为空

其他约束

auto_increment  自动增加，默认步长为1，一张表只能有一个自动增加字段

default  默认值

***添加与删除主键约束***

alter table 表名 modify 列名 类型（长度） primary key

alter table 表名 add constraint 约束名 primary key(列名);

alter table 表名 drop primary key; 主键上如果存在auto_increment,则删除不了


***添加与删除外键约束***

创建表在最后一列之后添加

constraint 约束名 foreign key(外键列名) references 主表名(主键列名)


为已经存在的表添加

alter table  表名 add constraint 约束名 foreign key(外键列名) references 主表(主键列名); 

删除外键约束

alter table 表名 drop foreign key 约束名

外键的级联操作

ON DELETE CASCASE  删除主表数据时，从表数据也随之删除

ON UPDATE CASCADE 更新主表数据是，从表数据也随之更新

ON DELETE SET NULL  删除主表中的数据时，从表中的数据置为空

***添加与删除唯一约束***

alter table 表名 add [constraint 约束名] unique(字段名[,字段名2，...])

alter table 表名 drop index 约束名

### 9.分组与聚合
聚合函数
sum()  max()  min()  count()  avg()

select sum(cnt) as cnt ,count(cnt) as cnt,sum(order_money) as order_money from order_view;

分组

group by  分组语句

将指定列的相同的值放在一个组中


select
count(*) orders,
sum(cnt) as sum_cnt,
sum(order_money) as order_money,
max(order_money) as maxorder_money,
min(order_money) as minorder_money
from order_view
group by fruit_name
having count(*) >= 2
and sum(order_money) >= 100；

sql 语句语句的注释用#

执行为source ~/sql/count_order.sql  source +路径

### 10.mysql与Python交互

pip install pymysql  安装python操作mysql的工具包

import pymysql  导入pymysl包

***连接数据库***

db  = pymysql.connect(host="127.0.0.1",user="root",  password="123456",db="test",port=3306)  

获取操作数据库的游标   cursor  = cur = db.cursor()  

***查询操作***

cursor.execute('select * from person') 
results =  cursor.fetchall()    #获取查询结果
for row in results :  
        print(row)
        
***插入操作***

db.autocommit(True) ＃ 自动提交事务
cursor.execute("insert into person values(1,'disen')") 
db.commit() 提交

exceute()也可执行创建和修改库与表语句

在执行插入、更新和删除语句时，可以用try-except语句，友好地处理错误。

使用execute(sql, args) 方法
cursor.execute("insert into person values(%d,%s)", (1, 'disen')) 
db.commit() 提交

***更新操作***

cursor.execute("update person set name='jack' where id = 1")  
db.commit()  

***删除操作***

 cur.execute("delete from person where id=1") 